# subway

In [1]:
import pandas as pd
import numpy as np
import datetime
import urllib
import os
import json
os.environ['KAKAO_KEY'] = '9c266cb030a36f29f9b6ea9875afa2d6'
os.environ['DATASEOUL_KEY'] = '504661694b79757933354551754c70'

from datetime import datetime

from collections import defaultdict

from christmas.crawl import NaverCrawler
from christmas.crawl import AddressCoordCrawler, SubwayAddressCrawler, WeatherHistoryCrawler


addr_coord_crawl = AddressCoordCrawler()
subway_addr_crawl = SubwayAddressCrawler()
weather_history_crawl = WeatherHistoryCrawler()

def drop_col(pd, columns):
    for col in columns:
        if col in pd.columns:
            pd = pd.drop(col, axis=1)
    return pd

In [ ]:
subway_list_path = './rawdata/subway/list/서울특별시 노선별 지하철역 정보(신규).csv'
subway_list_df = pd.read_csv(subway_list_path)

columns = ['code', 'name', 'eng_name', 'line', 'out_code']

subway_list_df.columns = columns

# subway_list_df.to_csv('./gendata/subway/list.csv', encoding='utf-8')
subway_list_df = subway_list_df.set_index(subway_list_df.name)
subway_list_df = subway_list_df.drop('name', axis=1)
print(subway_list_df.head())

In [ ]:
# 주소 정보
subway_addr_path_1 = './rawdata/subway/서울교통공사 지하철 역별 주소 정보.csv'
subway_addr_path_2 = './rawdata/subway/서울교통공사 지하철역 주소 및 전화번호 정보.csv'

subway_addr_df_1 = pd.read_csv(subway_addr_path_1)
subway_addr_df_2 = pd.read_csv(subway_addr_path_2)

subway_addr_df_1 = subway_addr_df_1.drop('연번', axis=1)
subway_addr_df_1 = subway_addr_df_1.drop('전화번호', axis=1)
subway_addr_df_1 = subway_addr_df_1.drop('지번주소', axis=1)
subway_addr_df_1 = subway_addr_df_1.drop('호선', axis=1)
subway_addr_df_1 = subway_addr_df_1.drop('우편번호', axis=1)

subway_addr_df_1.columns = ['name', 'address']

subway_addr_df_1.name = subway_addr_df_1.name.apply(lambda n: n.split('역')[0])

subway_addr_df_2 = subway_addr_df_2.drop('호선', axis=1)
subway_addr_df_2 = subway_addr_df_2.drop('구주소', axis=1)
subway_addr_df_2 = subway_addr_df_2.drop('전화번호', axis=1)

subway_addr_df_2.columns = ['name', 'address']

subway_addr_df = pd.concat([subway_addr_df_1, subway_addr_df_2])

del subway_addr_df_1, subway_addr_df_2

subway_addr_df = subway_addr_df.set_index(subway_addr_df.name)
subway_addr_df = subway_addr_df.drop('name', axis=1)

print(subway_addr_df.head())

In [ ]:
subway_df = subway_list_df.join(subway_addr_df)
print(subway_df.head())

In [ ]:
for idx, row in subway_df.iterrows():
    addr = row.address
    if not pd.isna(addr):
        x, y = addr_coord_crawl(addr)
        if not x:
            subway_df.at[idx, 'x'] = x
            subway_df.at[idx, 'y'] = y
            continue
    
    station_name = '{}역'.format(row.name)
    addr, x, y = subway_addr_crawl(station_name)
    if addr:
        subway_df.at[idx, 'address'] = addr
    subway_df.at[idx, 'x'] = x
    subway_df.at[idx, 'y'] = y

In [ ]:
subway_df.to_csv('./gendata/subway/list.csv', encoding='utf-8')
subway_df.head()

In [ ]:
subway_famous_path = './rawdata/subway/서울교통공사 역출구별 관광지 정보.csv'
subway_famous_pd = pd.read_csv(subway_famous_path)
subway_famous_pd = drop_col(subway_famous_pd, ['비고', 'URL', '호선', '역명', '일련번호'])

subway_famous_pd.columns = ['famous', 'code', 'famous_exit']

subway_famous_pd.code = subway_famous_pd.code.apply(lambda c: '%04d' % c)

def exit_to_num(exit):
    if pd.isnull(exit):
        return None
    
    if 'Exit' in exit:
        exit_num = exit.split('Exit ')[-1].strip()
        exit_num = exit_num.split(' ')[0]
        return int(exit_num)
    
    if '번출구' in exit:
        exit_num = exit.split('번출구')[0].strip()
        exit_num = exit_num.split(' ')[0]
        return int()
    
    if exit == '―':
        return None
    
    if '번 출구' in exit:
        return int(exit.split('번 출구')[0].strip())
    
    if '번' in exit:
        return int(exit.split('번')[0].strip())
    
    if exit.isnumeric():
        return int(exit)
    
    return None

subway_famous_pd.famous_exit = subway_famous_pd.famous_exit.apply(exit_to_num)


subway_known_path = './rawdata/subway/서울시 역코드로 지하철 주변 주요시설 정보 검색.csv'
subway_known_pd = pd.read_csv(subway_known_path, encoding='euc-kr', delimiter='\t')
subway_known_pd = drop_col(subway_known_pd, ['호선', '전철역명', '호선명', '외부코드', '일련번호', '전화번호', '홈페이지', '우편번호', '주소', 'X좌표', 'Y좌표', '툴팁', 'VR 경로', '아이콘', '출력여부'])
subway_known_pd.columns = ['code', 'famous_exit', 'famous']

def sanitize_exit_num(exit_num):
    if exit_num.isnumeric():
        return int(exit_num)
    return None

subway_known_pd.famous_exit = subway_known_pd.famous_exit.apply(sanitize_exit_num)

subway_places_pd = pd.concat([subway_famous_pd, subway_known_pd], sort=True)

print(subway_places_pd.head())

subway_places_pd.to_csv('./gendata/subway/places.csv', encoding='utf-8')

In [ ]:
# https://www.google.com/maps/search/?api=1&query=37.48666213075966,126.83879911291903

In [6]:
subway_people_path_1 = './rawdata/subway/people/CARD_SUBWAY_MONTH_201512.csv'
subway_people_path_2 = './rawdata/subway/people/CARD_SUBWAY_MONTH_201612.csv'
subway_people_path_3 = './rawdata/subway/people/CARD_SUBWAY_MONTH_201712.csv'
subway_people_df_1 = pd.read_csv(subway_people_path_1, encoding='euc-kr')
subway_people_df_2 = pd.read_csv(subway_people_path_2, encoding='euc-kr')
subway_people_df_3 = pd.read_csv(subway_people_path_3, encoding='utf-8', index_col=None)

subway_people_df = pd.concat([subway_people_df_1, subway_people_df_2, subway_people_df_3])

subway_people_df.columns = ['date', 'line', 'code', 'name', 'in_people', 'out_people', '_']
subway_people_df = subway_people_df.drop('_', axis=1)
# subway_people_df.date = subway_people_df.date.apply(lambda d: datetime.datetime.strptime(str(d), '%Y%m%d'))
# print(subway_people_df.dtypes)
subway_people_df.head()

subway_people_df.to_csv('./gendata/subway/in_out_by_date.csv', encoding='utf-8')

In [7]:
subway_people_df_3

,사용일자,노선명,역ID,역명,승차총승객수,하차총승객수,등록일자
20171201,경부선,1722,서정리,6052.0,5683.0,20171204,NaN
20171201,2호선,224,서초,26536.0,26469.0,20171204,NaN
20171201,중앙선,1207,양정,899.0,785.0,20171204,NaN
20171201,중앙선,1208,덕소,7199.0,7200.0,20171204,NaN
20171201,중앙선,1209,도심,3193.0,2465.0,20171204,NaN
20171201,중앙선,1210,팔당,705.0,680.0,20171204,NaN
20171201,중앙선,1211,운길산,754.0,647.0,20171204,NaN
20171201,중앙선,1212,양수,1972.0,2018.0,20171204,NaN
20171201,중앙선,1213,신원,238.0,180.0,20171204,NaN
20171201,중앙선,1214,국수,1069.0,1001.0,20171204,NaN


In [ ]:
def get_december_data(year):
    subway_people_time_path = './rawdata/subway/time/{}년 일별 역별 시간대별 승하차인원(1~8호선).xlsx'.format(year)
    subway_people_time_df = pd.read_excel(subway_people_time_path, sheet_name='{}년'.format(year))
    subway_people_time_df = drop_col(subway_people_time_df, ['역명', '호선', '월', '합 계'])
    subway_people_time_df.columns = [
        'date', 'code', 'type',
        '05~06', '06~07', '07~08', '08~09', '09~10', '10-11', '11~12',
        '12~13', '13~14', '14~15', '15~16', '16~17', '17~18', '18~19',
        '19~20', '20~21', '21~22', '22~23', '23~24', '24~'
    ]
    subway_people_time_df = subway_people_time_df.set_index(subway_people_time_df.date)
    subway_people_time_df = subway_people_time_df.drop('date', axis=1)
    subway_people_time_df = subway_people_time_df.query('"{year}-12-01" <= date <= "{next}-01-01"'.format(year=year, next=year+1))
    return subway_people_time_df

subway_people_time_df_list = []

subway_people_time_df_list.append(get_december_data(2009))
subway_people_time_df_list.append(get_december_data(2010))
subway_people_time_df_list.append(get_december_data(2011))
subway_people_time_df_list.append(get_december_data(2012))
subway_people_time_df_list.append(get_december_data(2013))
subway_people_time_df_list.append(get_december_data(2014))
subway_people_time_df_list.append(get_december_data(2015))
subway_people_time_df_list.append(get_december_data(2016))

In [ ]:
subway_people_time_path_17 = './rawdata/subway/time/2017년 일별 역별 시간대별 승하차인원(1~8호선).xlsx'
subway_people_time_df_17 = pd.read_excel(subway_people_time_path_17, sheet_name='2017년')
subway_people_time_df_17 = drop_col(subway_people_time_df_17, ['역명', '호선', '월', '합 계'])
subway_people_time_df_17.columns = [
    'date', 'code', 'type',
    '05~06', '06~07', '07~08', '08~09', '09~10', '10-11', '11~12',
    '12~13', '13~14', '14~15', '15~16', '16~17', '17~18', '18~19',
    '19~20', '20~21', '21~22', '22~23', '23~24', '24~'
]

def date_to_str(date):
    if isinstance(date, datetime):
        return date.strftime('%Y-%m-%d')
    return date

subway_people_time_df_17.date = subway_people_time_df_17.date.apply(date_to_str)
subway_people_time_df_17 = subway_people_time_df_17.query('"2017-12-01" <= date <= "2018-01-01"')
subway_people_time_df_17 = subway_people_time_df_17.set_index(subway_people_time_df_17.date)
subway_people_time_df_17 = subway_people_time_df_17.drop('date', axis=1)
subway_people_time_df_list.append(subway_people_time_df_17)

In [ ]:
subway_people_time_df = pd.concat(subway_people_time_df_list)
subway_people_time_df.to_csv('./gendata/subway/in_out_by_time.csv', encoding='utf-8')

# weather

In [2]:
seoul_region_list = [
    '강남', '강동', '강북', '강서', '관악', '광진', '구로',
    '금천', '노원', '도봉', '동대문', '동작', '마포', '서대문',
    '서초', '성동', '성북', '송파', '양천', '영등포', '용산',
    '은평', '종로', '중구', '중랑'
]
columns=[
    'date', 'region', 'ta_avg', 'ta_max', 'ta_min', 'hd_avg', 'hd_max', 'hd_min', 'ws_avg',
    'ws_max', 'rn_sum' # temp, humidity, wind speed, rain
]
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [10]:
def get_weather_list(year, region):
    d = weather_history_crawl(year, region)
    d = json.loads(d)
    if 'DailyWeatherStation' not in d:
        print(year, region)
        return None
    d = d['DailyWeatherStation']
    weather_list = []
    for weather in d['row']:
        date = weather['SAWS_OBS_TM']
        if date >= '{}1201'.format(year):
            ta_avg = weather['SAWS_TA_AVG']
            ta_max = weather['SAWS_TA_MAX']
            ta_min = weather['SAWS_TA_MIN']
            hd_avg = weather['SAWS_HD_AVG']
            hd_max = weather['SAWS_HD_MAX']
            hd_min = weather['SAWS_HD_MIN']
            ws_avg = weather['SAWS_WS_AVG']
            ws_max = weather['SAWS_WS_MAX']
            rn_sum = weather['SAWS_RN_SUM']

            weather_day_dict = {
                'date': date, 'region': region, 'ta_avg': ta_avg, 'ta_max': ta_max, 'ta_min': ta_min, 'hd_avg': hd_avg,
                'hd_max': hd_max, 'hd_min': hd_min, 'ws_avg': ws_avg, 'ws_max': ws_max,
                'rn_sum': rn_sum,
            }
            weather_list.append(weather_day_dict)
    return weather_list

In [11]:
whole_weather_list = []
for year in years:
    for region in seoul_region_list:
        weather_list = get_weather_list(year, region)
        if weather_list:
            whole_weather_list.extend(weather_list)
weather_pd = pd.DataFrame(whole_weather_list, columns=columns)

weather_pd.to_csv('./gendata/weather/by_region.csv', encoding='utf-8')

2018 강서
